In [13]:
import os
import pandas as pd
import numpy as np

In [2]:
path = "T:/DCProjects/DataPortal/Data/Output"

In [6]:
inpath = "T:/Tableau/tableauLODES/Datasources"

In [4]:
cities = pd.read_csv(os.path.join(path,"or_city_blockIDs.csv"))

In [5]:
cities.head()

,City,Place,Latitude,Longitude,BlockIDs
0,Adair Village,"Adair Village city, OR",44.673529,-123.216959,"['410030102001158', '410030102001177', '410030..."
1,Adams,"Adams city, OR",45.767510,-118.564254,"['410599503003070', '410599503003081', '410599..."
2,Adrian,"Adrian city, OR",43.740351,-117.070510,"['410459707001248', '410459707001247', '410459..."
3,Albany,"Albany city, OR",44.630080,-123.093637,"['410030101005012', '410030101003007', '410030..."
4,Amity,"Amity city, OR",45.116107,-123.204701,"['410710310002058', '410710310002061', '410710..."


In [30]:
def str2list(x):
    x = [value[1:-1] for value in x[1:-1].split(', ')]
    return x

In [32]:
cities["BlockIDs"] = cities.BlockIDs.apply(str2list)

In [7]:
year = 2019

In [9]:
or_od = pd.read_csv(os.path.join(inpath, "or_od_main_JT01_"+ str(year) + ".csv"))

In [10]:
or_od.shape

(1534963, 13)

In [11]:
or_od.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,410019501001166,410019501002126,1,0,1,0,0,0,1,1,0,0,20211018
1,410019501001166,410019501002197,1,0,0,1,0,0,1,1,0,0,20211018
2,410019501001166,410019501002204,1,0,1,0,0,0,1,1,0,0,20211018
3,410019501001166,410019501003075,1,0,0,1,0,1,0,1,0,0,20211018
4,410019501001196,410019501001003,1,0,1,0,0,1,0,0,0,1,20211018


In [16]:
or_od.w_geocode.dtype

dtype('int64')

In [12]:
city_blockID_lookup = {city:blockID for city, blockID in zip(cities.City, cities.BlockIDs)}

In [14]:
def lookup_geocode(code, lookup):
    for city,blockIDs in lookup.items():
        if code in blockIDs:
            return city
    return np.nan

In [33]:
block_city_map = {}
for city,blockIDs in zip(cities.City, cities.BlockIDs):
    for blockID in blockIDs:
        block_city_map[blockID] = city

In [34]:
for i,k in enumerate(block_city_map.keys()): 
    print(k, block_city_map[k])
    if i == 5:
        break

410030102001158 Adair Village
410030102001177 Adair Village
410030102001175 Adair Village
410030102001181 Adair Village
410030102001176 Adair Village
410030102001179 Adair Village


In [19]:
for col in ["w_geocode", "h_geocode"]:
    or_od[col] = or_od[col].astype(str)

In [21]:
lookup_geocode(or_od.w_geocode.values[0], city_blockID_lookup)

'Sumpter'

In [36]:
block_city_map.get(or_od.w_geocode.values[0], np.nan)

'Sumpter'

In [35]:
#or_od["w_city"] = or_od.w_geocode.apply(lambda geocode: lookup_geocode(geocode, city_blockID_lookup))
or_od["w_city"] = or_od.w_geocode.apply(lambda geocode: block_city_map.get(geocode, np.nan))

In [38]:
or_od["h_city"] = or_od.h_geocode.apply(lambda geocode: block_city_map.get(geocode, np.nan))

In [39]:
or_od.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,w_city,h_city
0,410019501001166,410019501002126,1,0,1,0,0,0,1,1,0,0,20211018,Sumpter,NaN
1,410019501001166,410019501002197,1,0,0,1,0,0,1,1,0,0,20211018,Sumpter,Haines
2,410019501001166,410019501002204,1,0,1,0,0,0,1,1,0,0,20211018,Sumpter,Haines
3,410019501001166,410019501003075,1,0,0,1,0,1,0,1,0,0,20211018,Sumpter,NaN
4,410019501001196,410019501001003,1,0,1,0,0,1,0,0,0,1,20211018,Sumpter,NaN


In [40]:
or_od_wh = or_od.loc[(or_od.w_city.notnull() & or_od.h_city.notnull()), :]

In [41]:
or_od_wh.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,w_city,h_city
1,410019501001166,410019501002197,1,0,0,1,0,0,1,1,0,0,20211018,Sumpter,Haines
2,410019501001166,410019501002204,1,0,1,0,0,0,1,1,0,0,20211018,Sumpter,Haines
7,410019501001196,410019504003026,1,1,0,0,0,1,0,0,0,1,20211018,Sumpter,Baker City
10,410019501001198,410019501001187,1,0,1,0,0,1,0,0,1,0,20211018,Sumpter,Sumpter
14,410019501001198,410019504001025,1,0,0,1,0,1,0,0,0,1,20211018,Sumpter,Baker City


In [47]:
or_od_agg = or_od_wh.groupby(['h_city', 'w_city']).agg({'S000': 'sum'})

In [61]:
or_od_agg.reset_index(inplace=True)

In [63]:
vars = ["City", "Place", "Latitude", "Longitude"]

In [65]:
hdf = or_od_agg.merge(cities[vars], left_on = 'h_city', right_on='City')

In [67]:
hdf.rename(columns={"S000":"TotalJobs", 
                    "City": "HomeCity",
                    "Place": "HmPlace", 
                    "Latitude":"HmCityLat", 
                    "Longitude":"HmCityLon"}, 
          inplace=True)

In [68]:
wdf = hdf.merge(cities[vars], left_on = 'w_city', right_on='City')

In [69]:
wdf.rename(columns={"City": "WorkCity",
                    "Place": "WkPlace", 
                    "Latitude":"WkCityLat", 
                    "Longitude":"WkCityLon"}, 
          inplace=True)

In [71]:
wdf.drop(columns=['h_city', 'w_city'], inplace=True)

In [72]:
targetvars = ["WorkCity", "WkPlace", "WkCityLat", "WkCityLon", "HomeCity" , 
                  "HmPlace", "HmCityLat", "HmCityLon", "TotalJobs"]

In [73]:
df = wdf[targetvars]

In [74]:
df.head()

,WorkCity,WkPlace,WkCityLat,WkCityLon,HomeCity,HmPlace,HmCityLat,HmCityLon,TotalJobs
0,Adair Village,"Adair Village city, OR",44.673529,-123.216959,Adair Village,"Adair Village city, OR",44.673529,-123.216959,4
1,Adair Village,"Adair Village city, OR",44.673529,-123.216959,Albany,"Albany city, OR",44.630080,-123.093637,15
2,Adair Village,"Adair Village city, OR",44.673529,-123.216959,Astoria,"Astoria city, OR",46.186886,-123.822933,1
3,Adair Village,"Adair Village city, OR",44.673529,-123.216959,Baker City,"Baker City city, OR",44.776755,-117.832218,1
4,Adair Village,"Adair Village city, OR",44.673529,-123.216959,Brookings,"Brookings city, OR",42.062022,-124.290770,1


In [75]:
df.to_csv(os.path.join(inpath, "CityTable.csv"), index=False)